<a href="https://colab.research.google.com/github/jdasam/aat3020-2023/blob/main/notebooks/4_Machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Translation

In [1]:
# install HuggingFace
!pip install transformers tokenizers

# If you are not using Colab, install below to read xlsx file
# !pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.4 MB/s eta 0:00:00


In [2]:
'''
Download dataset (originally from NIA AI-Hub)
'''

# !gdown 1CpsqOuuuB3I_PG5DbuqH1ssCFVerU46g
# !unzip -q nia-aihub-korean-english.zip

!gdown 1GMFWREWBVcD5vJdwNFadHmzVStclcyKf
!unzip -q nia-aihub-korean-english-txt.zip

Downloading...
From: https://drive.google.com/uc?id=1GMFWREWBVcD5vJdwNFadHmzVStclcyKf
To: /content/nia-aihub-korean-english-txt.zip
100% 449M/449M [00:05<00:00, 74.8MB/s]


In [ ]:
from pathlib import Path
import pandas as pd


In [ ]:
# You don't have to run this
'''
dataset_dir = Path('nia_korean_english')
data_list = sorted(list(dataset_dir.glob('*.xlsx')))
for path in data_list:
  df = pd.read_excel(path)
  kor_text_path = path.parent / (path.stem+'_kor.txt') 
  eng_text_path = path.parent / (path.stem+'_eng.txt') 
  with open(kor_text_path, 'w', encoding='utf8') as f:
      f.write('\n'.join(df['원문']))
  with open(eng_text_path, 'w', encoding='utf8') as f:
      f.write('\n'.join(df['번역문']))
'''


"\ndataset_dir = Path('nia_korean_english')\ndata_list = sorted(list(dataset_dir.glob('*.xlsx')))\nfor path in data_list:\n  df = pd.read_excel(path)\n  kor_text_path = path.parent / (path.stem+'_kor.txt') \n  eng_text_path = path.parent / (path.stem+'_eng.txt') \n  with open(kor_text_path, 'w', encoding='utf8') as f:\n      f.write('\n'.join(df['원문']))\n  with open(eng_text_path, 'w', encoding='utf8') as f:\n      f.write('\n'.join(df['번역문']))\n"

In [ ]:
dataset_dir = Path('nia_korean_english')
data_list = sorted(list(dataset_dir.glob('*.xlsx')))

data_list

[PosixPath('nia_korean_english/1_구어체(1).xlsx'),
 PosixPath('nia_korean_english/1_구어체(2).xlsx'),
 PosixPath('nia_korean_english/2_대화체.xlsx'),
 PosixPath('nia_korean_english/3_문어체_뉴스(1)_200226.xlsx'),
 PosixPath('nia_korean_english/3_문어체_뉴스(2).xlsx'),
 PosixPath('nia_korean_english/3_문어체_뉴스(3).xlsx'),
 PosixPath('nia_korean_english/3_문어체_뉴스(4).xlsx'),
 PosixPath('nia_korean_english/4_문어체_한국문화.xlsx'),
 PosixPath('nia_korean_english/5_문어체_조례.xlsx'),
 PosixPath('nia_korean_english/6_문어체_지자체웹사이트.xlsx')]

In [ ]:
import pandas as pd

dfs = [pd.read_excel(path) for path in data_list]

/home/teo/.local/share/virtualenvs/aat3020-2023-E1AG9i7b/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/teo/.local/share/virtualenvs/aat3020-2023-E1AG9i7b/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/teo/.local/share/virtualenvs/aat3020-2023-E1AG9i7b/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/teo/.local/share/virtualenvs/aat3020-2023-E1AG9i7b/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Wor

In [ ]:
df = pd.concat(dfs, axis=0)

In [ ]:
with open("nia_korean_english/1_구어체(1)_kor.txt", 'w', encoding='utf8') as f:
    f.write('\n'.join(df['원문']))


In [ ]:
df['원문'][10000:10050], df['번역문'][10000:10050]

## Huggingface Tokenizer

In [ ]:
from tokenizers import BertWordPieceTokenizer
tokenizer = BertWordPieceTokenizer(strip_accents=False, lowercase=False)

num_files = len(data_list)

corpus_file   =  [str(path.parent / (path.stem + '_kor.txt')) for path in data_list[:num_files]]
# output_dir   = Path('hugging_kor_%d'%(vocab_size))
en_corpus_file   =  [str(path.parent / (path.stem + '_eng.txt')) for path in data_list[:num_files]]
# output_dir   = Path('hugging_eng_%d'%(vocab_size))

vocab_size    = 32000  # Number of maximum size of the vocabulary
limit_alphabet= 6000   
output_dir    = Path('hugging_kor_%d'%(vocab_size))
en_output_dir = Path('hugging_eng_%d'%(vocab_size))
output_dir.mkdir(exist_ok=True)
en_output_dir.mkdir(exist_ok=True)
min_frequency = 5 

tokenizer.train(files=corpus_file,
               vocab_size=vocab_size,
               min_frequency=min_frequency,
               limit_alphabet=limit_alphabet, 
               show_progress=True)

tokenizer.save_model(str(output_dir))

en_tokenizer = BertWordPieceTokenizer(strip_accents=False, lowercase=False)
en_tokenizer.train(files=en_corpus_file,
                vocab_size=vocab_size,
                min_frequency=min_frequency,
                limit_alphabet=limit_alphabet,
                show_progress=True)
en_tokenizer.save_model(str(en_output_dir))



['hugging_eng_32000/vocab.txt']

In [3]:
from transformers import BertTokenizerFast

tokenizer_src = BertTokenizerFast.from_pretrained('hugging_kor_32000',
                                                       strip_accents=False,
                                                       lowercase=False) 
tokenizer_tgt = BertTokenizerFast.from_pretrained('hugging_eng_32000',
                                                       strip_accents=False,
                                                       lowercase=False) 

tokenized_data = tokenizer_src(df['원문'].iloc[10])
print(df['원문'].iloc[10])
print(tokenizer_src.decode(tokenized_data['input_ids']))

NameError: ignored

In [ ]:
tokenized_data['input_ids']

In [4]:
tokenized_ids = tokenizer_src("나는 서강대학교에 다닙니다")['input_ids']
tokenized_ids

[2, 6481, 25257, 8055, 4330, 2537, 10839, 3]

In [5]:
tokenizer_src.decode(tokenized_ids)

'[CLS] 나는 서강대학교에 다닙니다 [SEP]'

## Divide Train / Validate/ Test Set
- using `np.random.choice`
    - To always get the same random shuffling result, you have to use `np.random.seed()`

In [ ]:
df

,ID,지자체,원문,번역문
0,261811,경기도,"""경기도가 말산업 육성을 위해 총예산 245,193천원으로 2013년 경기도 용인시...","""The Gyeonggi provincial government announced ..."
1,409852,경기도,"""경기도가 주최하고 경기FTA활용지원센터와 코트라가 주관한 이번 시장개척단은 지난 ...","""Organized by Gyeonggi provincial government a..."
2,352671,경기도,"""경기도가 주최하고 경기도비정규직지원센터가 주관한 이번 교육은 공공 부문이 직·간접...","""Organized by Gyeonggi provincial government a..."
3,308191,경기도,"""경기도가 주최하고 사단법인 한국장애인복지시설협회 경기도협회(대표자:정권)에서 주관...","""Organized by Gyeonggi provincial government a..."
4,185424,경기도,"""경기도가 주최하고 인구보건복지협회 경기지회, 아이낳기좋은세상 경기운동본부가 주관하...","""Hosted by Gyeonggi provincial government, org..."
...,...,...,...,...
100082,73961,서울시 중구,힐튼호텔 건너편에서 회현동주민센터로 내려가는 길에는 칙칙한 회색의 석축옹벽이 이어져...,"Across from the Hilton Hotel, the path to Hoeh..."
100083,462950,경기도,힘든 분들이 많이 계시지만 조금이나마 어렵지 않도록 도움이 될 수 있도록 저희가 최...,There are a lot of people who are having a har...
100084,333006,경기도,힘든 역사 속에서 대한민국을 불과 50여년 만에 빛나는 나라로 만들고 업적을 만들 ...,It was possible to make the Republic of Korea ...
100085,454399,경기도,힘든 일을 하는 데는 무엇보다 정부가 큰 관심을 갖고 있다는 자부심을 갖도록 해야 한다.,"As for doing hard work, most of all, they shou..."


In [ ]:
len(df)

1602418

In [ ]:
df.iloc[-1]

ID                                                507679
지자체                                                  경기도
원문     힘을 합쳐 세계 일류통일강국으로 대한민국을 우뚝 세우는 역할을 이 자리의 모든 지도...
번역문    I would like to thank all the leaders here for...
Name: 100086, dtype: object

In [7]:
import pandas as pd
df = pd.read_excel("/content/nia_korean_english/1_구어체(1).xlsx")

## Define Dataset
- Each datasample has to return source sentence and target sentence
- You need a Tokenizer to get the tokenized result


In [8]:
import torch

class Dataset:
  def __init__(self, df, src_tokenizer, tgt_tokenizer):
    self.data = df
    self.src_tokenizer = src_tokenizer
    self.tgt_tokenizer = tgt_tokenizer
  
  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    selected_row = self.data.iloc[idx]
    source = selected_row['원문']
    target = selected_row['번역문']

    source_enc = self.src_tokenizer(source)['input_ids']
    target_enc = self.tgt_tokenizer(target)['input_ids']

    return torch.LongTensor(source_enc), torch.LongTensor(target_enc[:-1]), torch.LongTensor(target_enc[1:])
  
dataset = Dataset(df, tokenizer_src, tokenizer_tgt)

source, target, shifted_target = dataset[0]
print(source)
print(target)
print(shifted_target)

tensor([    2,    11,    70,  4665,  5209, 13306,    71, 12901,  9565, 12435,
           11,  3546, 14567,  4325,  8934,  8407,  7400,  4154,  3252,  6420,
        12985,  4996,  3397,  6461,    18,     3])
tensor([    2, 26268, 23067,    11,  1056,    69, 23067,  2803,  1067,  5155,
         1117,  1042,  2405,  4024,  5520,  1039,  1023, 26268,    18])
tensor([26268, 23067,    11,  1056,    69, 23067,  2803,  1067,  5155,  1117,
         1042,  2405,  4024,  5520,  1039,  1023, 26268,    18,     3])


## Split Dataset
- using ``torch.utils.data.random_split(dataset)``

In [9]:
num_data = len(dataset)
# num_data
num_train = int(num_data * 0.8)
num_valid = int(num_data * 0.1)
# num_test = int(num_data * 0.1)
num_test = num_data - (num_train + num_valid)

train_set, valid_set, test_set = torch.utils.data.random_split(dataset, [num_train, num_valid, num_test])
print(len(train_set), len(valid_set), len(test_set))

160000 20000 20000


In [10]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [12]:
train_loader = DataLoader(train_set, batch_size=4, shuffle=True)

batch = next(iter(train_loader))
batch

RuntimeError: ignored

## Define Collate function
- After implementing Dataset, we have to declare DataLoader that groups several training samples as a single batch
- However, we cannot batchify the melodies in straightforward way, because the length of each melody is different
- In this problem, you will learn about how to handle sequences of different length as a batch


In [ ]:

'''
This cell will make error, because the length of each sample is different to each other
'''

train_loader = DataLoader(train_set, batch_size=8)
batch = next(iter(train_loader))


In [18]:
'''
To handle that problem, you have to make your collate function 
'''
def your_collate_function(raw_batch):
  '''
  You can make your own function to handle the batch
  '''
  src = [x[0] for x in raw_batch]
  tgt = [x[1] for x in raw_batch]
  shifted_tgt = [x[2] for x in raw_batch]
  
  return src, tgt, shifted_tgt # This returns the first sample of each batch. So it will avoid the error, but it doesn't do proper batchifying

batch_size = 8
raw_batch = [train_set[i] for i in range(batch_size)] # This is the input for the collate function
batch = your_collate_function(raw_batch)

'''
This is what the 'collate_fn' does in DataLoader
'''

train_loader = DataLoader(train_set, batch_size=batch_size, collate_fn=your_collate_function)
batch_by_loader = next(iter(train_loader))
batch_by_loader

([tensor([    2,  7110,  7249,  6455, 14377, 18637, 15237,  8251, 25509,  4337,
          18905, 13630,  9107,  9135,    18,     3]),
  tensor([    2,  6832,  9081, 15420, 27164, 11105, 17601,  6412,  3160,  3252,
           6414,    18,     3]),
  tensor([    2,  3600,  6741, 12614,  6678, 28240,  3191,  4657,  4336, 12553,
          15888,    18,     3]),
  tensor([    2,  6730, 15523, 28173,  6625,  7334,  6507,    18,     3]),
  tensor([    2,  8543,  7009, 16955, 23111,  7637, 22306,    18,     3]),
  tensor([    2,  2284, 19570,  2426, 20003,  4325,  8756,  4421, 10833,  8419,
             18,     3]),
  tensor([    2, 30025,  7175,  2714,  4563, 12935,  6749,  7323, 20796,  4331,
           4173,  4417,  4694,    16,  3967,  4540,  4770,  7576, 15011,  7616,
          30310, 16615,  7280,  4421, 11548,  9583,  7023,    18,     3]),
  tensor([    2,  6481, 26684,  4330,  6642,  9186,  8398, 11003,  8827,  3301,
          12469, 10145,  4387,  6582,  3309,  4352, 28808,    18,    

In [16]:
raw_batch = [train_set[0], train_set[1], train_set[2]] # this is a sample input for collate function
raw_batch

[(tensor([    2,  7110,  7249,  6455, 14377, 18637, 15237,  8251, 25509,  4337,
          18905, 13630,  9107,  9135,    18,     3]),
  tensor([    2,  1287,  1109, 13814,  1042,  1065,  4666,  1061,    69,  1434,
             17,  2323,  8814,  1562,  1169, 18279,  2481,  1061,  4393,  2032,
             18]),
  tensor([ 1287,  1109, 13814,  1042,  1065,  4666,  1061,    69,  1434,    17,
           2323,  8814,  1562,  1169, 18279,  2481,  1061,  4393,  2032,    18,
              3])),
 (tensor([    2,  6832,  9081, 15420, 27164, 11105, 17601,  6412,  3160,  3252,
           6414,    18,     3]),
  tensor([   2, 1089,   11,   87, 1161, 1863, 1042, 1652, 1089, 7485, 1693, 1287,
          1195, 2834, 1089, 1039, 1023, 1521,   18]),
  tensor([1089,   11,   87, 1161, 1863, 1042, 1652, 1089, 7485, 1693, 1287, 1195,
          2834, 1089, 1039, 1023, 1521,   18,    3])),
 (tensor([    2,  3600,  6741, 12614,  6678, 28240,  3191,  4657,  4336, 12553,
          15888,    18,     3]),
  tensor

In [15]:
batch_by_loader # [train_set[0], train_set[1], train_set[2]]

[(tensor([    2,  7110,  7249,  6455, 14377, 18637, 15237,  8251, 25509,  4337,
          18905, 13630,  9107,  9135,    18,     3]),
  tensor([    2,  1287,  1109, 13814,  1042,  1065,  4666,  1061,    69,  1434,
             17,  2323,  8814,  1562,  1169, 18279,  2481,  1061,  4393,  2032,
             18]),
  tensor([ 1287,  1109, 13814,  1042,  1065,  4666,  1061,    69,  1434,    17,
           2323,  8814,  1562,  1169, 18279,  2481,  1061,  4393,  2032,    18,
              3])),
 (tensor([    2,  6832,  9081, 15420, 27164, 11105, 17601,  6412,  3160,  3252,
           6414,    18,     3]),
  tensor([   2, 1089,   11,   87, 1161, 1863, 1042, 1652, 1089, 7485, 1693, 1287,
          1195, 2834, 1089, 1039, 1023, 1521,   18]),
  tensor([1089,   11,   87, 1161, 1863, 1042, 1652, 1089, 7485, 1693, 1287, 1195,
          2834, 1089, 1039, 1023, 1521,   18,    3])),
 (tensor([    2,  3600,  6741, 12614,  6678, 28240,  3191,  4657,  4336, 12553,
          15888,    18,     3]),
  tensor

#### Pad Sequence and Pack Sequence
In PyTorch, there are two ways to batchify a group of sequence with different length.
- `torch.nn.utils.rnn.pad_sequence`
    - This function takes list of tensors with different length and return padded sequence
    - Padding is adding some constant number as a PAD token to match the length of short sequence to the maximum length
        - e.g. If there are sequence of length (3,7,4), we can add 4 zeros to sequence with length 3, 3 zeros to sequence with length 4 to make them length 7
    - In default, we use 0 for padding (zero padding)
    - The result 
- `torch.nn.utils.rnn.pack_sequence`
    - pad_sequence 

Cells below show the example of `pad_sequence`

In [19]:
from torch.nn.utils.rnn import pad_sequence, pack_sequence, PackedSequence
short = torch.arange(3, -1, -1).float() # [3, 2, 1, 0]
long = torch.arange(27,19, -1).float()
middle = torch.arange(15,9, -1).float()

pad_sequence([short, long, middle], batch_first=False)  # T x N 

tensor([[ 3., 27., 15.],
        [ 2., 26., 14.],
        [ 1., 25., 13.],
        [ 0., 24., 12.],
        [ 0., 23., 11.],
        [ 0., 22., 10.],
        [ 0., 21.,  0.],
        [ 0., 20.,  0.]])

In [20]:
# Default value of batch_first in pad_sequence is False.
# So you have to always be careful not to miss batch_first=True in pad_sequence, if you use batch_first=True for your RNN layer.
pad_sequence([short, long, middle], batch_first=True)  # N x T 

tensor([[ 3.,  2.,  1.,  0.,  0.,  0.,  0.,  0.],
        [27., 26., 25., 24., 23., 22., 21., 20.],
        [15., 14., 13., 12., 11., 10.,  0.,  0.]])

1) However, the problem is that you can't figure out whether the 0 at the end of each sequence is a padded one, or was included in the original sequence
- e.g. `[2, 3, 4, 3, 0]` becomes `[ 2,  3,  4,  5,  0,  0,  0]`. Now we don't know how many zeros were added for padding

2) Also, if you run RNN for this padded sequence, RNN will calculate for the padded part also.
- RNN doesn't know whether it is padded data, or existing data
- This makes computation slower

3) If you want to use bi-directional, which also reads the sequence from backward, paddings can make the result different.

To solve this issue, we use PackedSequence, by using `pack_sequence`/

In [21]:
packed_sequence = pack_sequence([short, long, middle], enforce_sorted=False)
packed_sequence

PackedSequence(data=tensor([27., 15.,  3., 26., 14.,  2., 25., 13.,  1., 24., 12.,  0., 23., 11.,
        22., 10., 21., 20.]), batch_sizes=tensor([3, 3, 3, 3, 2, 2, 1, 1]), sorted_indices=tensor([1, 2, 0]), unsorted_indices=tensor([2, 0, 1]))

`PackedSequence` has `data` and `batch_sizes`
- `data` contains the flattened value of given batch
    - To optimize the computation, the sequences have to be sorted by descending of length
- `batch_sizes` represents how many valid batch sample exists for each time step
    - `[3, 3, 3, 2, 2, 1, 1]` means that there are 3 sequences for first three time steps, and then 2 sequences for next two steps, and then only 1 sequence for next two steps.
- `sorted_indices` shows how the sorted sequences can be converted to original order.
    - `[1,2,0]` means that 
        - the 0th sequence in the sorted sequences (the longest one) was indexed as 1 in the original input batch
        - the 1st sequence in the sorted sequences (`middle`) was indexed as 2 in the original input batch
        - the 2nd sequence in the sorted sequences (`short`) was index as 0 in the original input batch
- `unsorted_indices` shows how the original sequences are sorted.
    - `[2,0,1]` means that
        - the 0th sequence in the original input was sorted as 2nd in the sorted sequences

In [22]:
rnn_layer = nn.GRU(1, 1)
packed_sequence = pack_sequence([short.unsqueeze(1), long.unsqueeze(1), middle.unsqueeze(1)], enforce_sorted=False)
out, last_hidden = rnn_layer(packed_sequence)

print(f"Type of output of RNN for PackedSequence: {type(out)}")
print(f"Type of last_hidden of RNN for PackedSequence: {type(last_hidden)}")

Type of output of RNN for PackedSequence: <class 'torch.nn.utils.rnn.PackedSequence'>
Type of last_hidden of RNN for PackedSequence: <class 'torch.Tensor'>


- RNN or its family of PyTorch can automatically handle `PackedSequence`
- However, other layers like `nn.Embedding` or `nn.Linear` cannot take `PackedSequence` as its input
- There are two ways to feed `PackedSequence` to these layers
    - First, convert PackedSequence to ordinary torch.Tensor by `torch.nn.utils.rnn.pad_packed_sequence`
        - This will convert PackedSequence to a tensor of sequneces with same length but different padding
    - The other way is to feed only PackedSequence.data, and then declaring new PackedSequence with the output as `data`

In [24]:
packed_sequence

PackedSequence(data=tensor([[27.],
        [15.],
        [ 3.],
        [26.],
        [14.],
        [ 2.],
        [25.],
        [13.],
        [ 1.],
        [24.],
        [12.],
        [ 0.],
        [23.],
        [11.],
        [22.],
        [10.],
        [21.],
        [20.]]), batch_sizes=tensor([3, 3, 3, 3, 2, 2, 1, 1]), sorted_indices=tensor([1, 2, 0]), unsorted_indices=tensor([2, 0, 1]))

In [23]:
'''
This will make error, because other layers cannot handle PackedSequence
'''
test_linear_layer = nn.Linear(in_features=1, out_features=2)
test_linear_layer(packed_sequence)

TypeError: ignored

In [26]:
'''
One way to to this is using torch.nn.utils.rnn.pad_packed_sequence to convert PackedSequence to ordinary padded tensor
'''

from torch.nn.utils.rnn import pad_packed_sequence
padded_sequence, batch_lengths = pad_packed_sequence(packed_sequence)
print(f'The padded sequence generated from packed sequence (squeezed for printing): \n {padded_sequence.squeeze()}')
print(f'"pad_packed_sequence" also returns "batch_lengths", to clarify the original length before the padding: \n {batch_lengths}')



The padded sequence generated from packed sequence (squeezed for printing): 
 tensor([[ 3., 27., 15.],
        [ 2., 26., 14.],
        [ 1., 25., 13.],
        [ 0., 24., 12.],
        [ 0., 23., 11.],
        [ 0., 22., 10.],
        [ 0., 21.,  0.],
        [ 0., 20.,  0.]])
"pad_packed_sequence" also returns "batch_lengths", to clarify the original length before the padding: 
 tensor([4, 8, 6])


In [27]:
'''
Now you can feed padded sequence to linear layer.
'''

linear_output = test_linear_layer(padded_sequence)
print(f"Output of feeding padded_sequence to a linear layer: {linear_output}")
print("Caution that it returns non-zero values for timestep with zero padding, because linear layer has a bias")

Output of feeding padded_sequence to a linear layer: tensor([[[ 3.3898,  1.8743],
         [24.5948, 12.2365],
         [13.9923,  7.0554]],

        [[ 2.5063,  1.4426],
         [23.7113, 11.8048],
         [13.1088,  6.6237]],

        [[ 1.6227,  1.0108],
         [22.8277, 11.3730],
         [12.2252,  6.1919]],

        [[ 0.7392,  0.5791],
         [21.9442, 10.9413],
         [11.3417,  5.7602]],

        [[ 0.7392,  0.5791],
         [21.0606, 10.5095],
         [10.4581,  5.3284]],

        [[ 0.7392,  0.5791],
         [20.1771, 10.0777],
         [ 9.5746,  4.8966]],

        [[ 0.7392,  0.5791],
         [19.2936,  9.6460],
         [ 0.7392,  0.5791]],

        [[ 0.7392,  0.5791],
         [18.4100,  9.2142],
         [ 0.7392,  0.5791]]], grad_fn=<ViewBackward0>)
Caution that it returns non-zero values for timestep with zero padding, because linear layer has a bias


In [28]:
'''
You can make the output as a PackedSequence, by using torch.nn.utils.rnn.pack_padded_sequence
'''
from torch.nn.utils.rnn import pack_padded_sequence
re_packed_sequence = pack_padded_sequence(linear_output, batch_lengths, enforce_sorted=False)
re_packed_sequence

PackedSequence(data=tensor([[24.5948, 12.2365],
        [13.9923,  7.0554],
        [ 3.3898,  1.8743],
        [23.7113, 11.8048],
        [13.1088,  6.6237],
        [ 2.5063,  1.4426],
        [22.8277, 11.3730],
        [12.2252,  6.1919],
        [ 1.6227,  1.0108],
        [21.9442, 10.9413],
        [11.3417,  5.7602],
        [ 0.7392,  0.5791],
        [21.0606, 10.5095],
        [10.4581,  5.3284],
        [20.1771, 10.0777],
        [ 9.5746,  4.8966],
        [19.2936,  9.6460],
        [18.4100,  9.2142]], grad_fn=<PackPaddedSequenceBackward0>), batch_sizes=tensor([3, 3, 3, 3, 2, 2, 1, 1]), sorted_indices=tensor([1, 2, 0]), unsorted_indices=tensor([2, 0, 1]))

In [25]:
'''
Another way to do it is using PackedSequence.data
'''

linear_out_pack = test_linear_layer(packed_sequence.data)
packed_sequence_after_linear = PackedSequence(linear_out_pack, packed_sequence.batch_sizes, packed_sequence.sorted_indices, packed_sequence.unsorted_indices)
packed_sequence_after_linear

PackedSequence(data=tensor([[24.5948, 12.2365],
        [13.9923,  7.0554],
        [ 3.3898,  1.8743],
        [23.7113, 11.8048],
        [13.1088,  6.6237],
        [ 2.5063,  1.4426],
        [22.8277, 11.3730],
        [12.2252,  6.1919],
        [ 1.6227,  1.0108],
        [21.9442, 10.9413],
        [11.3417,  5.7602],
        [ 0.7392,  0.5791],
        [21.0606, 10.5095],
        [10.4581,  5.3284],
        [20.1771, 10.0777],
        [ 9.5746,  4.8966],
        [19.2936,  9.6460],
        [18.4100,  9.2142]], grad_fn=<AddmmBackward0>), batch_sizes=tensor([3, 3, 3, 3, 2, 2, 1, 1]), sorted_indices=tensor([1, 2, 0]), unsorted_indices=tensor([2, 0, 1]))

In [29]:
def pack_collate(raw_batch):
  source, target, shifted_target = zip(*raw_batch)
  return pack_sequence(source, enforce_sorted=False), pack_sequence(target, enforce_sorted=False), pack_sequence(shifted_target, enforce_sorted=False)

train_loader = DataLoader(train_set, batch_size=8, collate_fn=pack_collate)
batch = next(iter(train_loader))
batch

(PackedSequence(data=tensor([    2,     2,     2,     2,     2,     2,     2,     2, 30025,  6481,
          7110,  6832,  3600,  2284,  6730,  8543,  7175, 26684,  7249,  9081,
          6741, 19570, 15523,  7009,  2714,  4330,  6455, 15420, 12614,  2426,
         28173, 16955,  4563,  6642, 14377, 27164,  6678, 20003,  6625, 23111,
         12935,  9186, 18637, 11105, 28240,  4325,  7334,  7637,  6749,  8398,
         15237, 17601,  3191,  8756,  6507, 22306,  7323, 11003,  8251,  6412,
          4657,  4421,    18,    18, 20796,  8827, 25509,  3160,  4336, 10833,
             3,     3,  4331,  3301,  4337,  3252, 12553,  8419,  4173, 12469,
         18905,  6414, 15888,    18,  4417, 10145, 13630,    18,    18,     3,
          4694,  4387,  9107,     3,     3,    16,  6582,  9135,  3967,  3309,
            18,  4540,  4352,     3,  4770, 28808,  7576,    18, 15011,     3,
          7616, 30310, 16615,  7280,  4421, 11548,  9583,  7023,    18,     3]), batch_sizes=tensor([8, 8, 8, 8

## Define Model
![image](https://raw.githubusercontent.com/tensorflow/nmt/master/nmt/g3doc/img/seq2seq.jpg)

In [ ]:
class Seq2seq(nn.Module):
  def __init__(self, enc_vocab, dec_vocab, hidden_size):
    super().__init__()
    self.encoder = Encoder(enc_vocab, hidden_size)
    self.decoder = Decoder(dec_vocab, hidden_size)

  def forward(self, src, tgt):
    enc_out = self.encoder(src)
    dec_out = self.decoder(tgt, enc_out)
    return dec_out 

class Encoder(nn.Module):
  def __init__(self, num_vocab, hidden_size, num_layers=2):
    super().__init__()
    self.emb = nn.Embedding(num_vocab, hidden_size)
    self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True, num_layers=num_layers)
    # batch_first True: it takes (Num_samples_in_batch, num_timesteps, num_dim)
    # batch_first False: it takes (num_timesteps, Num_samples_in_batch, num_dim)

  def forward(self, x):
    if isinstance(x, PackedSequence):
      emb = PackedSequence(self.emb(x.data), batch_sizes=x.batch_sizes, sorted_indices=x.sorted_indices, unsorted_indices=x.unsorted_indices)
    else:
      emb = self.emb(x)
    out, last_hidden = self.rnn(emb)
    return last_hidden


class Decoder(nn.Module):
  def __init__(self, num_vocab, hidden_size, num_layers=2):
    super().__init__()
    self.emb = nn.Embedding(num_vocab, hidden_size)
    self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True, num_layers=num_layers)
    self.proj = nn.Linear(hidden_size, num_vocab)


  def forward(self, x, enc_output):
    if isinstance(x, PackedSequence):
      emb = PackedSequence(self.emb(x.data), batch_sizes=x.batch_sizes, sorted_indices=x.sorted_indices, unsorted_indices=x.unsorted_indices)
      out, last_hidden = self.rnn(emb, enc_output)
      logit = PackedSequence(self.proj(out.data), batch_sizes=out.batch_sizes, sorted_indices=out.sorted_indices, unsorted_indices=out.unsorted_indices)
    else:
      emb = self.emb(x)
      out, last_hidden = self.rnn(emb, enc_output)
      logit = self.proj(out)
    return logit

hidden_size = 64
model = Seq2seq(tokenizer_src.vocab_size, tokenizer_tgt.vocab_size, hidden_size)
encoder = Encoder(tokenizer_src.vocab_size, hidden_size)
decoder = Decoder(tokenizer_tgt.vocab_size, hidden_size)

src, tgt, shifted_tgt = batch

enc_out = encoder(src)
dec_out = decoder(tgt, enc_out)

logit = model(src, tgt)
# logit.shape, logit.softmax(dim=-1)

In [ ]:
logit.shape, shifted_tgt.shape, shifted_tgt

(torch.Size([1, 10, 30679]),
 torch.Size([1, 10]),
 tensor([[  76,   10,  746,  810,  906,  335,  412, 2590,   17,    3]]))

In [ ]:
def nll_loss(pred, target, eps=1e-8):
  if pred.ndim == 3:
    pred = pred.flatten(0, 1)
  if target.ndim == 2:
    target = target.flatten(0, 1)
  assert pred.ndim == 2
  assert target.ndim == 1

  
  return -torch.log(pred[torch.arange(len(target)), target] + eps).mean()

# loss = nll_loss(logit.softmax(dim=-1), shifted_tgt)
# loss, loss.shape, shifted_tgt

## Define Trainer

In [ ]:
from tqdm.auto import tqdm

class Trainer:
  def __init__(self, model, optimizer, loss_fn, train_loader, valid_loader, device):
    self.model = model
    self.optimizer = optimizer
    self.loss_fn = loss_fn
    self.train_loader = train_loader
    self.valid_loader = valid_loader
    
    self.model.to(device)
    
    self.best_valid_accuracy = 0
    self.device = device
    
    self.training_loss = []
    self.validation_loss = []
    self.validation_acc = []

  def save_model(self, path='kor_eng_translator_model.pt'):
    torch.save({'model':self.model.state_dict(), 'optim':self.optimizer.state_dict()}, path)
    
  def train_by_num_epoch(self, num_epochs):
    for epoch in tqdm(range(num_epochs)):
      self.model.train()
      for batch in tqdm(self.train_loader, leave=False):
        loss_value = self._train_by_single_batch(batch)
        self.training_loss.append(loss_value)
      self.model.eval()
      validation_loss, validation_acc = self.validate()
      self.validation_loss.append(validation_loss)
      self.validation_acc.append(validation_acc)
      
      if validation_acc > self.best_valid_accuracy:
        print(f"Saving the model with best validation accuracy: Epoch {epoch+1}, Acc: {validation_acc:.4f} ")
        self.save_model('kor_eng_translator_model_best.pt')
      else:
        self.save_model('kor_eng_translator_model_last.pt')
      self.best_valid_accuracy = max(validation_acc, self.best_valid_accuracy)

      
  def _train_by_single_batch(self, batch):
    '''
    This method updates self.model's parameter with a given batch
    
    batch (tuple): (batch_of_input_text, batch_of_label)
    
    You have to use variables below:
    
    self.model (SentimentModel/torch.nn.Module): A neural network model
    self.optimizer (torch.optim.adam.Adam): Adam optimizer that optimizes model's parameter
    self.loss_fn (function): function for calculating BCE loss for a given prediction and target
    self.device (str): 'cuda' or 'cpu'

    output: loss (float): Mean binary cross entropy value for every sample in the training batch
    The model's parameters, optimizer's steps has to be updated inside this method

    TODO: Complete this method 
    '''
    src, tgt, shifted_tgt = batch
    src = src.to(self.device)
    tgt = tgt.to(self.device)
    shifted_tgt = shifted_tgt.to(self.device)

    logit = self.model(src, tgt)

    if isinstance(logit, PackedSequence):
      prob = logit.data.softmax(dim=-1)
      loss = self.loss_fn(prob, shifted_tgt.data)
    else:
      loss = self.loss_fn(prob, shifted_tgt)
    loss.backward()
    self.optimizer.step()
    self.optimizer.zero_grad

    return loss.item()

    
  def validate(self, external_loader=None):
    '''
    This method calculates accuracy and loss for given data loader.
    It can be used for validation step, or to get test set result
    
    input:
      data_loader: If there is no data_loader given, use self.valid_loader as default.
      
    
    output: 
      validation_loss (float): Mean Binary Cross Entropy value for every sample in validation set
      validation_accuracy (float): Mean Accuracy value for every sample in validation set
      
    TODO: Complete this method 

    '''
    
    ### Don't change this part
    if external_loader and isinstance(external_loader, DataLoader):
      loader = external_loader
      print('An arbitrary loader is used instead of Validation loader')
    else:
      loader = self.valid_loader
      
    self.model.eval()
    
    '''
    Write your code from here, using loader, self.model, self.loss_fn.
    '''

hidden_size = 128
model = Seq2seq(tokenizer_src.vocab_size, tokenizer_tgt.vocab_size, hidden_size)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nll_loss
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, collate_fn=pack_collate)
valid_loader = DataLoader(valid_set, batch_size=128, shuffle=False, collate_fn=pack_collate)
device = 'cuda'

trainer = Trainer(model, optimizer, loss_fn, train_loader, valid_loader, device)


In [ ]:
loss_records = []

for i in range(100):
  loss = trainer._train_by_single_batch(batch)
  loss_records.append(loss)
print(loss_records)

[10.386933326721191, 10.32657241821289, 10.268647193908691, 10.210345268249512, 10.149768829345703, 10.084954261779785, 10.013751029968262, 9.933724403381348, 9.842076301574707, 9.735479354858398, 9.609955787658691, 9.460831642150879, 9.283170700073242, 9.072586059570312, 8.827017784118652, 8.548736572265625, 8.245342254638672, 7.928269863128662, 7.609827041625977, 7.30034065246582, 7.006803512573242, 6.732766628265381, 6.479230880737305, 6.245960235595703, 6.0324931144714355, 5.8385162353515625, 5.663750171661377, 5.507776737213135, 5.369976997375488, 5.249426364898682, 5.144853115081787, 5.054716110229492, 4.977362632751465, 4.911262035369873, 4.85506534576416, 4.807579517364502, 4.767537593841553, 4.733421802520752, 4.70345401763916, 4.676103115081787, 4.650460720062256, 4.626137733459473, 4.6028008460998535, 4.580265998840332, 4.558821201324463, 4.539508819580078, 4.523560523986816, 4.511585235595703, 4.503292083740234, 4.497927665710449, 4.4949469566345215, 4.494128704071045, 4.49

In [ ]:
trainer.train_by_num_epoch(1)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/160000 [00:00<?, ?it/s]

In [ ]:
!gdown 10PR89HB2h2dKiXw-4GWQyhGvAOds3WKZ


In [ ]:
model = Seq2seq(tokenizer_src.vocab_size, tokenizer_tgt.vocab_size, hidden_size, num_layers=3)
